<a href="https://colab.research.google.com/github/profteachkids/CHE4071_Fall2023/blob/main/TransferFunctionAlgebra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots
import scipy as sp
from numpy.polynomial.polynomial import Polynomial as P
from scipy.signal import TransferFunction as TF

In [2]:
def tf2poly(tf):
    return P(tf.num[::-1]), P(tf.den[::-1])

def tf_mult(tf1,tf2):
    n1, d1 = tf2poly(tf1)
    n2, d2 = tf2poly(tf2)

    return TF((n1*n2).coef[::-1], (d1*d2).coef[::-1])

def tf_div(tf1,tf2):
    n1, d1 = tf2poly(tf1)
    n2, d2 = tf2poly(tf2)

    return TF((n1*d2).coef[::-1], (d1*n2).coef[::-1])

def tf_add(tf1, tf2):
    n1, d1 = tf2poly(tf1)
    n2, d2 = tf2poly(tf2)

    return TF((n1*d2 + n2*d1).coef[::-1], (d1*d2).coef[::-1])

def tf_feedback(tf_f, tf_b):
    nf, df = tf2poly(tf_f)
    nb, db = tf2poly(tf_b)
    return TF((nf*db).coef[::-1], (df*db + nf*nb).coef[::-1])

def pid(Kc, taui, taud):
    return TF([Kc*taud*taui, Kc*taui, Kc], [taui, 0])

In [46]:
def delay_pade(theta, n):
    f = 1
    c = [1]
    for i in range(1,n):
        f*=-theta/i
        c.append(f)
    p,q=sp.interpolate.pade(c, n//2)

    return TF(p.coef,q.coef)

In [69]:
tf_process = tf_mult(delay_pade(1,6),tf_mult(TF([1.],[2.,1.]), TF([1.],[2.,1.])))

In [72]:
Kc=1
taui=5
taud=0.
tf_closed_loop = tf_feedback(tf_mult(pid(Kc, taui, taud), tf_process), TF([1.],[1.]))

In [73]:
t=np.linspace(0,50,500)
x,y=tf_closed_loop.step(T=t)
fig=make_subplots()
fig.add_scatter(x=x,y=y)
fig.update_layout(width=800,height=600,template='plotly_dark')